In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import sklearn
import os
import time
import sys
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.17.4
pandas 0.25.3
sklearn 0.22
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [3]:
# tfrecord 文件格式
# -> tf.train.Example
#   -> tf.train.Features -> {'key': tf.train.Feature}
#       -> tf.train.Feature -> tf.train.ByteList/FloatList/Int64List


favoriate_books = [name.encode('utf-8') for name in ['machine learning', 'cc150']]
favoriate_books_bytelist = tf.train.BytesList(value = favoriate_books)
print(favoriate_books_bytelist)

hours_floatlist = tf.train.FloatList(value = [15.5, 9.5, 7.0, 8.0])
print(hours_floatlist)
age_int64list = tf.train.Int64List(value = [42])

features = tf.train.Features(
    feature = {
        'favoriate_books': tf.train.Feature(bytes_list = favoriate_books_bytelist),
        'hours': tf.train.Feature(float_list = hours_floatlist),
        'age': tf.train.Feature(int64_list = age_int64list)
    }
)

print(features)

value: "machine learning"
value: "cc150"

value: 15.5
value: 9.5
value: 7.0
value: 8.0

feature {
  key: "age"
  value {
    int64_list {
      value: 42
    }
  }
}
feature {
  key: "favoriate_books"
  value {
    bytes_list {
      value: "machine learning"
      value: "cc150"
    }
  }
}
feature {
  key: "hours"
  value {
    float_list {
      value: 15.5
      value: 9.5
      value: 7.0
      value: 8.0
    }
  }
}



In [4]:
example = tf.train.Example(features=features)
print(example)

serialized_example = example.SerializeToString()
print(serialized_example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "favoriate_books"
    value {
      bytes_list {
        value: "machine learning"
        value: "cc150"
      }
    }
  }
  feature {
    key: "hours"
    value {
      float_list {
        value: 15.5
        value: 9.5
        value: 7.0
        value: 8.0
      }
    }
  }
}

b'\n]\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n.\n\x0ffavoriate_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150'


In [11]:
output_dir = 'tfrecord_basic'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
filename = 'test.tfrecords'
file_fullpath = os.path.join(output_dir, filename)
with tf.io.TFRecordWriter(file_fullpath) as writer:
    for i in range(3):
        writer.write(serialized_example)

In [6]:
dataset = tf.data.TFRecordDataset([file_fullpath])
for serialized_example_tensor in dataset:
    print(serialized_example_tensor)

tf.Tensor(b'\n]\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n.\n\x0ffavoriate_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150', shape=(), dtype=string)
tf.Tensor(b'\n]\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n.\n\x0ffavoriate_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150', shape=(), dtype=string)
tf.Tensor(b'\n]\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n.\n\x0ffavoriate_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150', shape=(), dtype=string)


In [9]:
expected_features = {
    'favoriate_books': tf.io.VarLenFeature(dtype = tf.string),
    'hours': tf.io.VarLenFeature(dtype = tf.float32),
    'age': tf.io.FixedLenFeature([], dtype=tf.int64)
}
dataset = tf.data.TFRecordDataset([file_fullpath])
for serialized_example_tensor in dataset:
    example = tf.io.parse_single_example(
        serialized_example_tensor,
        expected_features
    )
    books = tf.sparse.to_dense(example['favoriate_books'], default_value=b'')
    for book in books:
        print(book.numpy().decode('UTF-8'))

machine learning
cc150
machine learning
cc150
machine learning
cc150


In [12]:
file_fullpath_zip = file_fullpath + '.zip'
options = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter(file_fullpath_zip, options) as writer:
    for i in range(3):
        writer.write(serialized_example)

In [13]:
dataset_zip = tf.data.TFRecordDataset([file_fullpath_zip], compression_type='GZIP')
for serialized_example_tensor in dataset_zip:
    example = tf.io.parse_single_example(
        serialized_example_tensor,
        expected_features
    )
    books = tf.sparse.to_dense(example['favoriate_books'], default_value=b'')
    for book in books:
        print(book.numpy().decode('UTF-8'))

machine learning
cc150
machine learning
cc150
machine learning
cc150
